In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

# class quatum bbs

In [ ]:
class Periodic_bbs:# p is the configuration of balls
    def __init__(s1elf, p):
        p = str(p)
        self.boxlength = len(p)
        p_array = np.zeros(len(p), dtype = int)
        for i in range(len(p)):
            p_array[i] = int(p[i])
        
        self.box = p_array
        self.carrier = []
        self.energy = 0
        self.time = 0
        self.log = []
    
    def combitional_R(self, vl, b):
        vl = np.sort(vl)
        if b == 1:
            if max(vl) == 1:
                vl = vl
                b_dash = b
            
            elif max(vl) == 2:
                vl[-1] = 1
                vl = np.sort(vl)
                b_dash = 2
            else:
                print("error")
        
        elif b == 2:
            if min(vl) == 2:
                vl = vl
                b_dash = b
            elif min(vl) == 1:
                vl[0] = 2
                vl = np.sort(vl)
                b_dash = 1
            else:
                print("error")
            
        return vl, b_dash
                
        
    
    def map_R(self, vl):
        p_dash = self.box.copy()
        for i in range(len(self.box)):
            b = self.box[i]
            vl, b_dash =  self.combitional_R(vl ,b)
            p_dash[i] = b_dash
        #self.box = p_dash
        return vl, p_dash
    
    def time_evolve(self, l, calc_energy = False, print_result = True):
        self.time += 1
        vl = np.ones(l)
        vl_new = np.zeros(l)
        p_temp = self.box.copy()
        #from IPython.core.debugger import Pdb; Pdb().set_trace()
        for i in range(l+100):
            vl = np.sort(vl)
            self.box = p_temp.copy()
            vl_new, p_dash = self.map_R(vl)
            if (vl != vl_new).any():
                vl[0] = 2
            elif (vl == vl_new).all():
                break
            else:
                print("error")
        if calc_energy == True:
            self.energy = np.count_nonzero((p_dash - p_temp) == -1)
        self.box = p_dash
        self.log.append(self.box)
        if print_result == True:
            if calc_energy == True:
                print("t = {0} | system = {1} | energy = {2}".format(self.time, self.box, self.energy))
            else:
                print("t = {0} | {1}".format(self.time, self.box))
        
        return vl_new, self.box
    
        
    
